# Robot Policy Control

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-35QaaoiXx6VDmVNBOEtmpLs9VBs)

This notebook demonstrates how to load and run a trained reinforcement learning policy on the Unitree Go2 quadruped robot. The policy was trained in IsaacLab using PhysX simulation.

Key topics:
- Loading pre-trained PyTorch policies
- Computing observations for policy input
- Understanding reward functions
- Running closed-loop control with MuJoCo solver


## Setup and Imports


In [ ]:
import newton
import warp as wp
import torch
import yaml
import numpy as np
from tqdm.notebook import trange

## Download Robot Assets

Newton provides utilities to download robot models and trained policies from the `newton-assets` repository.


In [ ]:
# Download Go2 robot assets (USD model, policy, config)
asset_directory = str(newton.utils.download_asset("unitree_go2"))

# Load configuration from YAML
config_path = f"{asset_directory}/rl_policies/go2.yaml"
with open(config_path, encoding="utf-8") as f:
    config = yaml.safe_load(f)

print(f"Number of DOFs: {config['num_dofs']}")
print(f"Action scale: {config['action_scale']}")

## Building the Model

We'll load the Go2 robot from USD and configure it for simulation with the MuJoCo solver.


In [ ]:
# Create model builder
builder = newton.ModelBuilder(up_axis=newton.Axis.Z)

# Register MuJoCo-specific attributes
newton.solvers.SolverMuJoCo.register_custom_attributes(builder)

# Configure joint defaults
builder.default_joint_cfg = newton.ModelBuilder.JointDofConfig(
    armature=0.1,
    limit_ke=1.0e2,
    limit_kd=1.0e0,
)

# Configure contact parameters
builder.default_shape_cfg.ke = 5.0e4  # Contact stiffness
builder.default_shape_cfg.kd = 5.0e2  # Contact damping
builder.default_shape_cfg.kf = 1.0e3  # Friction stiffness
builder.default_shape_cfg.mu = 0.75   # Friction coefficient

In [ ]:
# Import Go2 robot from USD
builder.add_usd(
    f"{asset_directory}/usd/go2.usda",
    xform=wp.transform(wp.vec3(0, 0, 0.8)),
    collapse_fixed_joints=False,
    enable_self_collisions=False,
    hide_collision_shapes=True,
)

# Approximate mesh colliders with convex hulls
builder.approximate_meshes("convex_hull")

# Add ground plane
builder.add_ground_plane()

print(f"Robot loaded: {builder.body_count} bodies, {builder.joint_count} joints")

### Configure Initial State and Joint Properties

Set the initial pose and configure joint stiffness/damping for PD control.


In [ ]:
# Set initial floating base pose (position + quaternion)
builder.joint_q[:3] = [0.0, 0.0, 0.76]  # Position
builder.joint_q[3:7] = [0.0, 0.0, 0.7071, 0.7071]  # Quaternion (facing forward)

# Set initial joint positions from config
builder.joint_q[7:] = config["mjw_joint_pos"]

# Configure joint PD gains and armature
for i in range(config['num_dofs']):
    builder.joint_target_ke[i + 6] = config["mjw_joint_stiffness"][i]
    builder.joint_target_kd[i + 6] = config["mjw_joint_damping"][i]
    builder.joint_armature[i + 6] = config["mjw_joint_armature"][i]

print("Initial state configured")


In [ ]:
# Finalize the model
model = builder.finalize()
model.set_gravity((0.0, 0.0, -9.81))

print(f"Model finalized: {model.body_count} bodies, {model.joint_count} joints")

## Create Solver and States

We use the MuJoCo solver for accurate articulated body dynamics.


In [ ]:
# Create MuJoCo solver
solver = newton.solvers.SolverMuJoCo(
    model,
    use_mujoco_cpu=False,
    solver="newton",
    nconmax=30,
    njmax=100,
)

# Create state objects
state_0 = model.state()
state_1 = model.state()
control = model.control()

# Evaluate forward kinematics
newton.eval_fk(model, state_0.joint_q, state_0.joint_qd, state_0)

print("Solver and states initialized")

## Load Pre-trained Policy

The policy is a neural network trained with reinforcement learning in IsaacLab. It maps observations to joint position targets.


In [ ]:
# Setup PyTorch device
device = wp.get_device()
torch_device = "cuda" if device.is_cuda else "cpu"

# Load policy
policy_path = f"{asset_directory}/rl_policies/physx_go2.pt"
policy = torch.jit.load(policy_path, map_location=torch_device)

print(f"Policy loaded from: {policy_path}")
print(f"Device: {torch_device}")


## Observation and Reward Functions

### Observation Space

The policy receives observations consisting of:
- Base linear velocity (body frame): 3D
- Base angular velocity (body frame): 3D
- Projected gravity vector: 3D
- Command (forward, lateral, yaw): 3D
- Joint positions (relative to default): 23D
- Joint velocities: 23D
- Previous actions: 23D

Total: **81 dimensions**

### Reward Function

The policy was trained to maximize rewards based on:

**Tracking rewards:**
- Linear velocity tracking: $r_{\text{lin}} = \exp(-||v_{xy} - v_{\text{cmd}}||^2 / \sigma)$
- Angular velocity tracking: $r_{\text{ang}} = \exp(-|\omega_z - \omega_{\text{cmd}}|^2 / \sigma)$

**Regularization penalties:**
- Action rate: $-||a_t - a_{t-1}||^2$ (smooth actions)
- Joint acceleration: $-||\ddot{q}||^2$ (smooth motion)
- Torque: $-||\tau||^2$ (energy efficiency)
- Base height: $-(z - z_{\text{target}})^2$ (maintain upright posture)
- Orientation: $-||q_{xy}||^2$ (stay upright)

The total reward is a weighted sum of these terms, with weights tuned during training.


## Observation Computation

Helper function to compute policy observations from simulation state.


In [ ]:
@torch.jit.script
def quat_rotate_inverse(q: torch.Tensor, v: torch.Tensor) -> torch.Tensor:
    """Rotate vector by inverse of quaternion (world to body frame)."""
    q_w = q[..., 3]
    q_vec = q[..., :3]
    a = v * (2.0 * q_w**2 - 1.0).unsqueeze(-1)
    b = torch.cross(q_vec, v, dim=-1) * q_w.unsqueeze(-1) * 2.0
    c = q_vec * torch.bmm(q_vec.view(q.shape[0], 1, 3), v.view(q.shape[0], 3, 1)).squeeze(-1) * 2.0
    return a - b + c


def compute_obs(
    state: newton.State,
    joint_pos_initial: torch.Tensor,
    actions: torch.Tensor,
    command: torch.Tensor,
    indices: torch.Tensor,
) -> torch.Tensor:
    """Compute observation vector for policy."""
    # Extract state (floating base + joints)
    root_quat = torch.tensor(state.joint_q[3:7], device=torch_device, dtype=torch.float32).unsqueeze(0)
    root_lin_vel = torch.tensor(state.joint_qd[:3], device=torch_device, dtype=torch.float32).unsqueeze(0)
    root_ang_vel = torch.tensor(state.joint_qd[3:6], device=torch_device, dtype=torch.float32).unsqueeze(0)
    joint_pos = torch.tensor(state.joint_q[7:], device=torch_device, dtype=torch.float32).unsqueeze(0)
    joint_vel = torch.tensor(state.joint_qd[6:], device=torch_device, dtype=torch.float32).unsqueeze(0)
    
    # Transform to body frame
    vel_b = quat_rotate_inverse(root_quat, root_lin_vel)
    ang_vel_b = quat_rotate_inverse(root_quat, root_ang_vel)
    gravity_vec = torch.tensor([0.0, 0.0, -1.0], device=torch_device, dtype=torch.float32).unsqueeze(0)
    grav_b = quat_rotate_inverse(root_quat, gravity_vec)
    
    # Compute relative joint positions and velocities
    joint_pos_rel = joint_pos - joint_pos_initial
    
    # Reorder joints to match policy training (PhysX ordering)
    joint_pos_rel = torch.index_select(joint_pos_rel, 1, indices)
    joint_vel = torch.index_select(joint_vel, 1, indices)
    
    # Concatenate observation
    obs = torch.cat([vel_b, ang_vel_b, grav_b, command, joint_pos_rel, joint_vel, actions], dim=1)
    
    return obs

## Initialize Policy Tensors

Setup joint mappings and initial tensors for policy execution.


In [ ]:
# Joint ordering mapping (PhysX training vs MuJoCo Warp)
def find_joint_mapping(mjw_names, physx_names):
    """Map between MuJoCo Warp and PhysX joint orderings."""
    mjw_to_physx = [physx_names.index(j) for j in mjw_names if j in physx_names]
    physx_to_mjw = [mjw_names.index(j) for j in physx_names if j in mjw_names]
    return mjw_to_physx, physx_to_mjw

mjw_to_physx, physx_to_mjw = find_joint_mapping(
    config["mjw_joint_names"],
    config["physx_joint_names"]
)

# Convert to torch tensors
physx_to_mjw_indices = torch.tensor(physx_to_mjw, device=torch_device, dtype=torch.long)
mjw_to_physx_indices = torch.tensor(mjw_to_physx, device=torch_device, dtype=torch.long)

# Initialize policy state
joint_pos_initial = torch.tensor(state_0.joint_q[7:], device=torch_device, dtype=torch.float32).unsqueeze(0)
actions = torch.zeros(1, config['num_dofs'], device=torch_device, dtype=torch.float32)
command = torch.tensor([[1.0, 0.0, 1.0]], device=torch_device, dtype=torch.float32)  # Walk forward

print(f"Policy tensors initialized")
print(f"Command: forward={command[0,0]:.1f}, lateral={command[0,1]:.1f}, yaw={command[0,2]:.1f}")

## Simulation Loop

Run closed-loop control: observe → policy → act → simulate.


In [ ]:
# Simulation parameters
fps = 200
frame_dt = 1.0 / fps
decimation = 4  # Policy runs at 50 Hz (200 / 4)
sim_substeps = 8  # Must be even for proper state swapping
sim_dt = frame_dt / sim_substeps

print(f"Simulation: {fps} Hz, Policy: {fps/decimation} Hz")
print(f"Substeps: {sim_substeps}, dt: {sim_dt:.6f} s")

In [ ]:
def simulate_step():
    global state_0, state_1
    """Run multiple physics substeps."""
    for i in range(sim_substeps):
        state_0.clear_forces()
        contacts = model.collide(state_0)
        solver.step(state_0, state_1, control, contacts, sim_dt)
        state_0, state_1 = state_1, state_0


def policy_step():
    """Compute policy action and update control."""
    global actions
    
    # Compute observation
    obs = compute_obs(state_0, joint_pos_initial, actions, command, physx_to_mjw_indices)
    
    # Run policy
    with torch.no_grad():
        actions = policy(obs)
    
    # Reorder actions to MuJoCo Warp joint ordering
    actions_mjw = torch.index_select(actions, 1, mjw_to_physx_indices)
    
    # Compute joint targets (default + scaled action)
    joint_targets = joint_pos_initial + config["action_scale"] * actions_mjw
    
    # Add zeros for floating base (6 DOF) and convert to Warp
    targets_with_base = torch.cat([
        torch.zeros(6, device=torch_device, dtype=torch.float32),
        joint_targets.squeeze(0)
    ])
    
    # Update control
    control.joint_target_pos.assign(wp.from_torch(targets_with_base, dtype=wp.float32))

## GPU Acceleration with CUDA Graphs

For maximum performance, capture the simulation loop as a CUDA graph to reduce kernel launch overhead.


In [ ]:
# Capture simulation as CUDA graph (if running on GPU)
graph = None
if wp.get_device().is_cuda and wp.is_mempool_enabled(wp.get_device()):
    print("Capturing CUDA graph for optimized execution...")
    with wp.ScopedCapture() as capture:
        simulate_step()
    graph = capture.graph
    print("CUDA graph captured successfully")
else:
    print("Running on CPU (no CUDA graph)")

## Run Simulation with Viewer

Execute the policy and visualize the robot walking forward. The CUDA graph accelerates the physics simulation significantly.


In [ ]:
# Create viewer
viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)

# Simulation loop
num_frames = 2500
sim_time = 0.0

for frame in trange(num_frames, desc="Simulating Go2 walking"):
    # Run policy every decimation steps
    if frame % decimation == 0:
        policy_step()
    
    # Physics step (use CUDA graph if available)
    if graph:
        wp.capture_launch(graph)
    else:
        simulate_step()
    
    # Log to viewer
    viewer.begin_frame(sim_time)
    viewer.log_state(state_0)
    viewer.end_frame()
    
    sim_time += frame_dt

viewer

## Summary

In this notebook, we:

1. **Loaded a pre-trained policy** from PyTorch JIT format
2. **Built the Go2 robot model** from USD with proper contact parameters
3. **Configured PD controllers** with joint stiffness and damping
4. **Computed observations** including base velocities, gravity, and joint states
5. **Understood reward functions** used during RL training
6. **Ran closed-loop control** with the MuJoCo solver
7. **Visualized results** in Rerun

### Key Concepts

- **Policy Network**: Maps observations → actions (joint position offsets)
- **PD Control**: Tracks policy targets with stiffness/damping gains
- **Joint Ordering**: PhysX training order vs MuJoCo Warp simulation order
- **Observation Space**: Body-frame velocities + proprioception + commands
- **Reward Shaping**: Balance tracking performance with regularization

### Next Steps

- Modify the command vector to control walking direction
- Experiment with different robots (Go2, Anymal), see `example_robot_policy.py` in the Newton examples
- Train your own policies using IsaacLab
- Add terrain or obstacles to test robustness
